## Introduction

Data scientists commonly spend over half their time cleaning data, so knowing how to clean "messy" data is an extremely important skill.

### Reading CSV Files with Encodings

We'll learn the basics of data cleaning with pandas as we work with `laptops.csv`, a CSV file containing information about 1,300 laptop computers.

Computers, at their lowest levels, can only understand binary - 0 and 1- and encodings are systems for representing characters in binary.
Something we can do if our file has an unknown encoding is to try the most common encodings:
- UTF-8
- Latin-1 (also known as ISO-8859-1)
- Windows-1251

In [1]:
import pandas as pd

laptops = pd.read_csv("data/laptops.csv", encoding='Latin-1')
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
Manufacturer                1303 non-null object
Model Name                  1303 non-null object
Category                    1303 non-null object
Screen Size                 1303 non-null object
Screen                      1303 non-null object
CPU                         1303 non-null object
RAM                         1303 non-null object
 Storage                    1303 non-null object
GPU                         1303 non-null object
Operating System            1303 non-null object
Operating System Version    1133 non-null object
Weight                      1303 non-null object
Price (Euros)               1303 non-null object
dtypes: object(13)
memory usage: 66.2+ KB


### Cleaning Column Names

